## 일베 댓글+ 캐글 데이터로 fast-text 임베딩 하기

In [3]:
import pandas as pd
import numpy as np
import re

from gensim.models import FastText
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:

df = pd.read_csv('../dataset/womad_swear.csv', index_col=0)
df2 = pd.read_csv('../dataset/비속어댓글총합_특수문자,null값제거o.csv')

In [5]:
df2

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
190576,드라마도 완전 재밌고 배우들도 멋있어서,0
190577,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
190578,케석대 어깨 올라간거봐라,1
190579,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


In [6]:
df2.columns=['msg', 'label']

In [7]:
df = pd.concat([df,df2])
print(df.shape)

(192581, 2)


In [8]:
df = df.astype('str')

In [9]:
df['msg']=df['msg'].str.replace(pat=r'[ㄱ-ㅎ]+', repl= r'', regex=True)

In [10]:
df["msg"] = df["msg"].str.replace('^ +', "") # white space 데이터를 empty value로 변경
df["msg"].replace('', np.nan, inplace=True)
print(df.isnull().sum())

msg      43
label     0
dtype: int64


In [11]:
df = df.dropna(how = 'any')
print(len(df))

192538


In [12]:
df

,msg,label
0,지금 어디 계세요,0
1,한국 시간에 시계 맞췄어,0
2,햄버거 두 개랑 콜라 주세요,0
3,우리는 밤새 춤추고 노래했다,0
4,무엇보다도 스트레스를 줄이는 것이 중요합니다,0
...,...,...
190576,드라마도 완전 재밌고 배우들도 멋있어서,0
190577,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
190578,케석대 어깨 올라간거봐라,1
190579,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


# 자모 분류, fast text 는 git 참고 
https://github.com/smothly/bad-word-detection/blob/master/FastTextVocab.ipynb

In [13]:

CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)


In [14]:
# 각 문장을 jamo_split하여 저장
#from JamoSplit import jamo_split, jamo_combine
df.loc[:,'msg'] = df.loc[:,'msg'].apply(lambda x: jamo_split(x))
df.loc[:,'msg'] = df.loc[:,'msg'].apply(lambda x: x.split(" "))

In [15]:
df.head(15)

,msg,label
0,"[ㅈㅣ_ㄱㅡㅁ, ㅇㅓ_ㄷㅣ_, ㄱㅖ_ㅅㅔ_ㅇㅛ_]",0
1,"[ㅎㅏㄴㄱㅜㄱ, ㅅㅣ_ㄱㅏㄴㅇㅔ_, ㅅㅣ_ㄱㅖ_, ㅁㅏㅈㅊㅝㅆㅇㅓ_]",0
2,"[ㅎㅐㅁㅂㅓ_ㄱㅓ_, ㄷㅜ_, ㄱㅐ_ㄹㅏㅇ, ㅋㅗㄹㄹㅏ_, ㅈㅜ_ㅅㅔ_ㅇㅛ_]",0
3,"[ㅇㅜ_ㄹㅣ_ㄴㅡㄴ, ㅂㅏㅁㅅㅐ_, ㅊㅜㅁㅊㅜ_ㄱㅗ_, ㄴㅗ_ㄹㅐ_ㅎㅐㅆㄷㅏ_]",0
4,"[ㅁㅜ_ㅇㅓㅅㅂㅗ_ㄷㅏ_ㄷㅗ_, ㅅㅡ_ㅌㅡ_ㄹㅔ_ㅅㅡ_ㄹㅡㄹ, ㅈㅜㄹㅇㅣ_ㄴㅡㄴ, ...",0
5,"[ㄷㅏㅇㅈㅏㅇ, ㅅㅣ_ㅈㅏㅇㄱㅏ_ㅅㅓ_, ㅃㅓㄹㄱㅓㅇㅅㅐㄱㅇㅡ_ㄹㅗ_, ㅅㅏ_ㅇㅘ_...",1
6,"[ㅅㅔ_ㅌㅡ_ㄹㅗ_, ㅍㅏㄴㄷㅏ_, ㅇㅜㅅㄱㅣ_ㄴㅗ_]",1
7,"[ㅇㅣ_, ㅅㅜ_ㅂㅏㄱ, ㅈㅏ_ㄹㅡ_ㄱㅔ_, ㅋㅏㄹ, ㅈㅗㅁ, ㄱㅏ_ㅈㅕ_ㅇㅘ_ㄹㅏ_]",0
8,"[ㅎㅜ_ㅍㅏㄹ, ㅁㅝㄴ, ㅅㅗ_ㄹㅣ_ㄴㅗ_, ㅇㅢ_ㄱㅕㄴㅇㅡㄹ, ㄴㅐ_ㄴㅡㄴㄱㅔ_,...",1
9,"[ㅁㅣ_ㄱㅜㄱ, ㅅㅏ_ㄹㅏㅁㄷㅡㄹㅇㅡㄴ, ㅈㅓㅅㄱㅏ_ㄹㅏㄱ, ㄷㅐ_ㅅㅣㄴ, ㅍㅗ_ㅋ...",0


In [16]:
df.label = df.label.astype('int64')

In [17]:
df.dtypes

msg      object
label     int64
dtype: object

In [18]:
# dataframe -> list로 변환
sentence_list = list(df['msg'])
len(sentence_list)

192538

In [19]:
# fasttext 인풋 완성!
sentence_list[:10]

[['ㅈㅣ_ㄱㅡㅁ', 'ㅇㅓ_ㄷㅣ_', 'ㄱㅖ_ㅅㅔ_ㅇㅛ_'],
 ['ㅎㅏㄴㄱㅜㄱ', 'ㅅㅣ_ㄱㅏㄴㅇㅔ_', 'ㅅㅣ_ㄱㅖ_', 'ㅁㅏㅈㅊㅝㅆㅇㅓ_'],
 ['ㅎㅐㅁㅂㅓ_ㄱㅓ_', 'ㄷㅜ_', 'ㄱㅐ_ㄹㅏㅇ', 'ㅋㅗㄹㄹㅏ_', 'ㅈㅜ_ㅅㅔ_ㅇㅛ_'],
 ['ㅇㅜ_ㄹㅣ_ㄴㅡㄴ', 'ㅂㅏㅁㅅㅐ_', 'ㅊㅜㅁㅊㅜ_ㄱㅗ_', 'ㄴㅗ_ㄹㅐ_ㅎㅐㅆㄷㅏ_'],
 ['ㅁㅜ_ㅇㅓㅅㅂㅗ_ㄷㅏ_ㄷㅗ_',
  'ㅅㅡ_ㅌㅡ_ㄹㅔ_ㅅㅡ_ㄹㅡㄹ',
  'ㅈㅜㄹㅇㅣ_ㄴㅡㄴ',
  'ㄱㅓㅅㅇㅣ_',
  'ㅈㅜㅇㅇㅛ_ㅎㅏㅂㄴㅣ_ㄷㅏ_'],
 ['ㄷㅏㅇㅈㅏㅇ',
  'ㅅㅣ_ㅈㅏㅇㄱㅏ_ㅅㅓ_',
  'ㅃㅓㄹㄱㅓㅇㅅㅐㄱㅇㅡ_ㄹㅗ_',
  'ㅅㅏ_ㅇㅘ_ㅅㅓ_',
  'ㅂㅗ_ㅈㅣ_ㄷㅏㅂㄱㅔ_',
  'ㅋㅡ_ㄱㅔ_',
  '',
  'ㄹㅗ_ㄱㅗ_',
  'ㅂㅏㄱㄱㅗ_',
  'ㅇㅣㅂㄱㅗ_ㄷㅏ_ㄴㅣ_ㄱㅔㅅㄴㅗ_'],
 ['ㅅㅔ_ㅌㅡ_ㄹㅗ_', 'ㅍㅏㄴㄷㅏ_', 'ㅇㅜㅅㄱㅣ_ㄴㅗ_'],
 ['ㅇㅣ_', 'ㅅㅜ_ㅂㅏㄱ', 'ㅈㅏ_ㄹㅡ_ㄱㅔ_', 'ㅋㅏㄹ', 'ㅈㅗㅁ', 'ㄱㅏ_ㅈㅕ_ㅇㅘ_ㄹㅏ_'],
 ['ㅎㅜ_ㅍㅏㄹ',
  'ㅁㅝㄴ',
  'ㅅㅗ_ㄹㅣ_ㄴㅗ_',
  'ㅇㅢ_ㄱㅕㄴㅇㅡㄹ',
  'ㄴㅐ_ㄴㅡㄴㄱㅔ_',
  'ㄷㅏㅇㅇㅕㄴㅎㅏㄴㄱㅓ_ㄴㅗ_ㅈㅏ_ㄷㅐㅇㅇㅣ_ㄴㅡㄴ',
  'ㅇㅓㄹㅁㅏ_ㄴㅏ_',
  'ㄱㅏ_ㅅㅡ_ㄹㅏ_ㅇㅣ_ㅌㅣㅇㅎㅏ_ㄱㅗ_',
  'ㅎㅜ_ㄹㅕ_ㅊㅣ_ㅁㅕ_',
  'ㅅㅏ_ㄴㅡㄴㄱㅓ_ㄴㅗ_'],
 ['ㅁㅣ_ㄱㅜㄱ',
  'ㅅㅏ_ㄹㅏㅁㄷㅡㄹㅇㅡㄴ',
  'ㅈㅓㅅㄱㅏ_ㄹㅏㄱ',
  'ㄷㅐ_ㅅㅣㄴ',
  'ㅍㅗ_ㅋㅡ_ㄹㅡㄹ',
  'ㅅㅏ_ㅇㅛㅇㅎㅏㄴㄷㅏ_']]

### FastText WordEmbedding

In [280]:
# 임베딩 차원: 50
# window size: 좌우 2단어 비속어는 좌우단어와 별로 연관이 없다고 판단...
# min_count: 최소 3번 등장한 단어들
# workers: -1 전부!!
# sg: skipgram이 더 성능이 좋기 때문
# min_n max_n : n-gram단위인데 한글자가 3글자라 최소 자모3개부터 최대 6개까지 ngram하기로 하였다. 1글자 ~ 2글자
# iter: 반복횟수 10

model = FastText(sentence_list, size=100, window=2, min_count=2, workers=4, sg=1, min_n=3, max_n=6, iter=10)

In [281]:
# 총 62105개 단어로 vocab이 만들어졌다.
# len(model.wv)
len(model.wv.vocab)

106481

In [282]:
# 연도 같은 경우는 연도와 비슷한 단어들이 나온다.
model.wv.most_similar(jamo_split("2018년"))

[('2017ㄴㅕㄴ', 0.9943025708198547),
 ('2019ㄴㅕㄴ', 0.991732656955719),
 ('2016ㄴㅕㄴ', 0.9901601672172546),
 ('18ㄴㅕㄴ', 0.9854506850242615),
 ('2011ㄴㅕㄴ', 0.9844149351119995),
 ('2012ㄴㅕㄴ', 0.9826508164405823),
 ('1972ㄴㅕㄴ', 0.9809393882751465),
 ('1978ㄴㅕㄴ', 0.9801497459411621),
 ('78ㄴㅕㄴ', 0.979729413986206),
 ('1988ㄴㅕㄴ', 0.97959965467453)]

In [283]:
# 욕설 같은 경우는 비슷한 형태의 욕설이 나온다.
model.wv.most_similar(jamo_split("개새끼"))

[('ㅉㅏㅇㄱㅐ_ㅅㅐ_ㄲㅣ_', 0.9809759259223938),
 ('ㄱㅐ_ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9803922176361084),
 ('ㅈㅟ_ㅅㅐ_ㄲㅣ_', 0.9798666834831238),
 ('ㄱㅐ_ㅆㅐ_ㄲㅣ_', 0.9782588481903076),
 ('ㄱㅐ_ㄷㅗㄱㅅㅐ_ㄲㅣ_', 0.9769392013549805),
 ('ㅇㅐ_ㅅㅐ_ㄲㅣ_', 0.9765238761901855),
 ('ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.975825309753418),
 ('ㅊㅣ_ㅁㅐ_ㅅㅐ_ㄲㅣ_', 0.9758047461509705),
 ('ㅅㅣㅂㅅㅐ_ㄲㅣ_', 0.975188136100769),
 ('ㅁㅜㄴㅅㅐ_ㄲㅣ_', 0.9731311202049255)]

In [284]:
model.wv.most_similar(jamo_split("시발"))

[('ㅅㅣ_ㅂㅏㄹㄹㅗㅁ', 0.9624711275100708),
 ('ㅁㅝㄴㅅㅣ_ㅂㅏㄹ', 0.962238609790802),
 ('ㅅㅣ_ㅂㅏㅋ', 0.9508090019226074),
 ('ㅆㅣ_ㅂㅏㄹ', 0.9504096508026123),
 ('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9465654492378235),
 ('ㅅㅣ_ㅂㅏㄹㄹㅓㅁ', 0.9379845261573792),
 ('ㅇㅘ_ㅅㅣ_ㅂㅏㄹ', 0.937768816947937),
 ('ㅇㅘ_ㅆㅣ_ㅂㅏㄹ', 0.935078501701355),
 ('ㅅㅣ_ㅂㅏㄹㄴㅗㅁ', 0.931434154510498),
 ('ㅂㅕㄹㅆㅣ_ㅂㅏㄹ', 0.9272646307945251)]

In [285]:
model.wv.most_similar(jamo_split("시바"))

[('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9265952706336975),
 ('ㅅㅣ_ㅂㅏㄹㄹㅗㅁ', 0.925764799118042),
 ('ㅅㅣ_ㅂㅏㅁ', 0.9249156713485718),
 ('ㅅㅣ_ㅂㅏㅋ', 0.9246200323104858),
 ('ㅅㅣ_ㅂㅏㄹㄹㅗㅁㅇㅏ_', 0.9242600798606873),
 ('ㅅㅣ_ㅂㅏㄹㄹㅓㅁㅇㅏ_', 0.9190875291824341),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴㅇㅏ_', 0.9140830039978027),
 ('ㅅㅣ_ㅂㅏㄹㅇㅏ_', 0.9128806591033936),
 ('ㅅㅣ_ㅂㅏㄱ', 0.912753701210022),
 ('ㅅㅣ_ㅂㅏㄹㄹㅓㅁ', 0.9107001423835754)]

In [286]:
model.wv.most_similar(jamo_split("미친"))

[('ㅁㅣ_ㅊㅣㄴㄹㅠ_', 0.9070913195610046),
 ('ㅁㅣ_ㅊㅣㄴㅅㅐㄲ', 0.8866625428199768),
 ('ㅁㅣ_ㅊㅣㄴㄴㅓㅁ', 0.885146975517273),
 ('ㅁㅣ_ㅊㅣㅁ', 0.8524152040481567),
 ('ㅁㅣ_ㅊㅣㄴㅈㅣㅅ', 0.8413803577423096),
 ('ㄱㅐ_ㅁㅣ_ㅊㅣㄴ', 0.8347471952438354),
 ('ㅁㅣ_ㅊㅣㄴㄴㅗㅁㅇㅏㅋ', 0.8325145840644836),
 ('ㅁㅣ_ㅊㅣㄴㄴㅗㅁ', 0.8316489458084106),
 ('ㅇㅝ_ㅊㅣㄴ', 0.8311556577682495),
 ('ㅁㅣ_ㅊㅣㄴㅅㅐ_ㅋㅣ_', 0.8179538249969482)]

In [287]:
model.wv.most_similar(jamo_split("존나"))

[('ㅈㅗㄴㄴㅏ_ㅍㅐㅁ', 0.9604917764663696),
 ('ㅈㅗㄴㄴㅏ_ㅆㅏㅁ', 0.9603818655014038),
 ('ㅇㅛㄱㅈㅗㄴㄴㅏ_', 0.9584428668022156),
 ('ㅈㅗㄴㄴㅏ_ㅂㅗㅁ', 0.9465975165367126),
 ('ㅈㅗㄴㄴㅏㅋ', 0.9295055270195007),
 ('ㅅㅗㄴㄴㅏ_', 0.9276456832885742),
 ('ㅇㅗㄴㄴㅏ_', 0.9250023365020752),
 ('ㅈㅗ_ㅎㅗㄴㄴㅏ_', 0.9158093929290771),
 ('ㅈㅗ_ㅇㅗㄴㄴㅏ_', 0.9084568619728088),
 ('ㅈㅗㄴㄴㅏ_ㅇㅜㅅㅈㅏ_', 0.8876283764839172)]

In [288]:
model.wv.most_similar(jamo_split("지랄"))

[('ㅈㅣ_ㄹㅏㄹㅈㅣ_ㄹㅏㄹ', 0.9808254241943359),
 ('ㅈㅣ_ㄹㅏㄹㄸㅓㅁ', 0.9746071100234985),
 ('ㅈㅣ_ㄹㅏㄹㅂㅕㅇ', 0.969560980796814),
 ('ㅈㅣ_ㄹㅏㄹㅈㅗㅁ', 0.9575825333595276),
 ('ㅈㅣ_ㄹㅏㄹㅂㅏㄹㄱㅘㅇ', 0.9508788585662842),
 ('ㅈㅣ_ㄹㅏㄹㅈㅣ_ㄹㅏㄹㅎㅏㅁ', 0.9425343871116638),
 ('ㅆㅣㅂㅈㅣ_ㄹㅏㄹ', 0.9376556277275085),
 ('ㅈㅗㅈㅈㅣ_ㄹㅏㄹ', 0.9363299608230591),
 ('ㅈㅣ_ㄹㅏㄹㅁㅏ_', 0.935675859451294),
 ('ㅈㅣ_ㄹㅏㄹㅇㅣㅁ', 0.9290183186531067)]

In [289]:
model.wv.most_similar(jamo_split("좆까"))

[('ㅈㅗㅅㄲㅏ_', 0.9598065614700317),
 ('ㅈㅗㅅㅉㅣㄴㄸㅏ_', 0.951840341091156),
 ('ㅈㅗㅈㅌㅏ_', 0.9494582414627075),
 ('ㅈㅗㅈㅉㅣㄴㄸㅏ_', 0.9483563899993896),
 ('ㅈㅗㅈㅂㅏㅂㅇㅏ_', 0.9469739198684692),
 ('ㅈㅗ_ㄲㅏ_', 0.9116729497909546),
 ('ㅆㅣㅂㅉㅣㄴㄸㅏ_', 0.9061601161956787),
 ('ㅆㅣㅂㅆㅣㅂㄸㅏ_', 0.8965063095092773),
 ('ㅉㅣㄴㄸㅏ_', 0.8927517533302307),
 ('ㅈㅗㅇㅅㅓㄱㅇㅏ_', 0.8923841714859009)]

In [290]:
model.wv.most_similar(jamo_split("페미"))

[('ㅂㅏㄴㅍㅔ_ㅁㅣ_', 0.9641053676605225),
 ('ㄲㅗㄹㅍㅔ_ㅁㅣ_', 0.9609299302101135),
 ('ㅅㅔ_ㅁㅣ_', 0.9361070990562439),
 ('ㅇㅑㅇㅋㅜ_ㅁㅣ_', 0.9283140897750854),
 ('ㅊㅟ_ㅁㅣ_', 0.9196064472198486),
 ('ㅂㅜ_ㅅㅔ_ㅁㅣ_', 0.9137302041053772),
 ('ㄴㅗ_ㅈㅗ_ㅁㅣ_', 0.9020801782608032),
 ('ㄱㅜ_ㅁㅣ_', 0.9017068147659302),
 ('ㄷㅔ_ㅁㅣ_', 0.8988704681396484),
 ('ㅊㅣㄴㅁㅣ_', 0.8945830464363098)]

In [291]:
# embedding model 저장
model.save("./festtext_embedded_2.model")

### ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [2]:
# fasttext 모델 불러오기

from gensim.models import FastText

embedded_model = FastText.load("./festtext_embedded_2.model")

In [20]:
# 각 단어를 벡터화 시켜주는 과정 3 x 100(embedding dimension) 
df['msg'] = df['msg'].apply(lambda x: [embedded_model[_] for _ in x])

/tmp/ipykernel_3183/377602601.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  df['msg'] = df['msg'].apply(lambda x: [embedded_model[_] for _ in x])


In [21]:
df.head()

,msg,label
0,"[[0.41951078, -0.511246, -1.5877508, 0.6485419...",0
1,"[[1.4425132, -0.7165865, 0.4393839, 0.63240397...",0
2,"[[-0.120705, 0.42407006, 0.074355334, -0.24042...",0
3,"[[-0.25409234, 0.07807473, -0.028147921, 0.346...",0
4,"[[-0.51335895, -0.15244816, -0.045989577, 0.43...",0


In [88]:
# 벡터화된 데이터로 저장
# df.to_json("./fasttext_vectorized_1.json")

## 패딩하기

In [22]:

df.columns = ["댓글", "비속어여부"]
df.head()

,댓글,비속어여부
0,"[[0.41951078, -0.511246, -1.5877508, 0.6485419...",0
1,"[[1.4425132, -0.7165865, 0.4393839, 0.63240397...",0
2,"[[-0.120705, 0.42407006, 0.074355334, -0.24042...",0
3,"[[-0.25409234, 0.07807473, -0.028147921, 0.346...",0
4,"[[-0.51335895, -0.15244816, -0.045989577, 0.43...",0


In [23]:
# 상위 20개 문장의 단어 개수 살펴보기
for i in range(0,20):
    print(len(df.iloc[i,0]))

3
4
5
4
5
10
3
6
10
6
8
4
1
3
5
5
4
5
2
97


In [24]:
print('문장의 최대 단어길이 :',max(len(review) for review in df['댓글']))
print('문장의 평균 단어길이 :',sum(map(len, df['댓글']))/len(df['댓글']))

# 한 문장의 최대 단어의 개수는 424개, 평균 단어의 개수는 8개

문장의 최대 단어길이 : 424
문장의 평균 단어길이 : 8.233881103989862


In [25]:
def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 문장들 중 단어개수가 %s 이하인 문장의 비율: %s'%(max_len, (count / len(nested_list))*100))
    
max_len = 35
below_threshold_len(max_len, df['댓글'])

# 98 % 가 35 단어 이하

전체 문장들 중 단어개수가 35 이하인 문장의 비율: 98.2616418577112


In [26]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df['댓글'], df['비속어여부'] , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

154030 154030 38508 38508


In [27]:
df['댓글'].head(10)

0    [[0.41951078, -0.511246, -1.5877508, 0.6485419...
1    [[1.4425132, -0.7165865, 0.4393839, 0.63240397...
2    [[-0.120705, 0.42407006, 0.074355334, -0.24042...
3    [[-0.25409234, 0.07807473, -0.028147921, 0.346...
4    [[-0.51335895, -0.15244816, -0.045989577, 0.43...
5    [[0.60792017, -0.605673, -0.6324978, 0.1391605...
6    [[-0.42568287, -0.86902803, 0.6506364, -0.0910...
7    [[-0.38475332, -0.7303187, 0.24930461, -0.8123...
8    [[-0.3624178, -0.32522255, -0.68778664, -0.474...
9    [[0.6328293, -1.460831, -0.11593121, 0.5354653...
Name: 댓글, dtype: object

In [28]:
train_x

103856    [[0.36108777, 0.6972935, -1.9984382, 1.16814, ...
103319    [[-0.30878735, -0.43910986, -0.092063986, 0.43...
126946    [[-0.3534226, -0.38606519, 0.041319467, 0.1720...
111954    [[-0.039441008, -0.39012972, 0.46631065, -0.15...
37327     [[-1.2736967, -0.73795694, 0.2720769, 0.398750...
                                ...                        
150358    [[-0.83524084, 0.21744843, -0.23513043, -0.554...
175006    [[-0.71190524, -0.46963233, -0.55986434, -0.07...
115995    [[0.07512964, 0.46976826, 0.41242698, -1.39595...
171728    [[-0.03165149, -0.029893627, -0.34336156, -0.0...
41610     [[-0.0813052, 0.013297408, -0.16820937, -0.251...
Name: 댓글, Length: 154030, dtype: object

In [29]:
train_y

103856    1
103319    1
126946    1
111954    0
37327     0
         ..
150358    0
175006    0
115995    1
171728    0
41610     0
Name: 비속어여부, Length: 154030, dtype: int64

In [30]:
# a=[]
# for i in train_x:
#     a.append(np.array(train_x))
# a

In [31]:
# y array형식으로 변경
y_train = np.array(train_y)
y_test = np.array(test_y)

In [32]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(train_x, maxlen=max_len)
X_test = pad_sequences(test_x, maxlen=max_len)

In [33]:
X_train[:]

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  1],
        [ 0, -1,  0, ...,  2,  2,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  0, ...,  0, -1,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ..., -1,  1,  0],
        [-2, -2,  0, ...,  0,  0,  0],
        [ 0,  1,  0, ...,  0, -1,  0]],

       ...,

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0, -1,  0],
        [ 0, -1,  0, ...,  0, -1,  0],
        [ 0,  0,  0, ...,  0,  0

In [34]:
print(X_test.shape)
print(X_train.shape)

(38508, 35, 100)
(154030, 35, 100)


In [35]:
print(y_test.shape)
print(y_train.shape)

(38508,)
(154030,)


In [36]:
y_test[:]

array([0, 0, 0, ..., 0, 0, 0])

### LSTM 학습하기 

In [37]:
from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(LSTM(units=1, input_shape=(35, 100)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
# mc = ModelCheckpoint('FT_best_model_v0.0.1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

2022-05-30 10:29:35.657041: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-30 10:29:35.699922: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-30 10:29:35.733701: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-30 10:29:35.833605: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-30 10:29:35.927027: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-30 10:29:35.931777: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

In [38]:

model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

2022-05-30 10:29:41.318573: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 2156420000 exceeds 10% of free system memory.


Epoch 1/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.3047 - accuracy: 0.8706 - val_loss: 0.2691 - val_accuracy: 0.8882
Epoch 2/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2667 - accuracy: 0.8857 - val_loss: 0.2636 - val_accuracy: 0.8900
Epoch 3/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2645 - accuracy: 0.8863 - val_loss: 0.2643 - val_accuracy: 0.8890
Epoch 4/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2637 - accuracy: 0.8869 - val_loss: 0.2632 - val_accuracy: 0.8891
Epoch 5/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2635 - accuracy: 0.8866 - val_loss: 0.2626 - val_accuracy: 0.8895
Epoch 6/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2632 - accuracy: 0.8867 - val_loss: 0.2626 - val_accuracy: 0.8892
Epoch 7/10
2407/2407 [==============================] - 19s 8ms/step - loss: 0.2631 - accuracy: 0.8864 - val_loss: 0.2633 - val_ac

In [39]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1])) # model.evaluate(X_test, y_test)
# x와 y 의 test 에 대한 정확도 출력

1204/1204 [==============================] - 2s 2ms/step - loss: 0.2618 - accuracy: 0.8897

 테스트 정확도: 0.8897


In [40]:
model.save("./fasttext_LSTM2.h5")

In [28]:
# model.save("./fasttext_model")

2022-05-10 09:01:21.980801: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./fasttext_model/assets


In [41]:
lstm_model = load_model('./fasttext_LSTM2.h5')

# 모델 사용해서 예측하기

### for index in range(len(test_word.split)) 일 경우 예측 - O

In [43]:
def test_result(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 1:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")


In [44]:
test_result("마라탕", embedded_model, lstm_model) # ㅠㅠ 

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


/tmp/ipykernel_3183/3614073675.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있지 않습니다.


In [45]:
test_result("지랄하지마", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_3183/3614073675.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [46]:
test_result("나가서 놀고싶다", embedded_model, lstm_model)

/tmp/ipykernel_3183/3614073675.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있지 않습니다.


### for index in range(len(s)) 일 경우 예측 - O

In [47]:
def test_result2(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] == 1:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [48]:
test_result2("너무 어려워요", embedded_model, lstm_model)

/tmp/ipykernel_3183/4136958889.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있지 않습니다.


In [49]:
test_result2("마라탕", embedded_model, lstm_model) # ?? ㅠㅠ

/tmp/ipykernel_3183/4136958889.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있지 않습니다.


In [50]:
test_result2("졸라 시끄러워", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_3183/4136958889.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### for index in range(len(test_word)) 일 경우 예측 - O

In [107]:
def test_result10(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [108]:
test_result10("졸라 시끄러워", embedded_model, lstm_model)

/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


In [109]:
test_result10("마라탕", embedded_model, lstm_model)

/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


In [111]:
test_result10("집에 갈꺼에요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [112]:
test_result10("집에 갈꺼에용", embedded_model, lstm_model) # 왜일까

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### 반대로 적용해보기 - X

In [88]:
def test_result3(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")

In [90]:
test_result3("졸라 시끄러워", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/3385605691.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [91]:
test_result3("돌체라떼 너무 맛있어요", embedded_model, lstm_model) # 반대로 하는거는 틀린것같음!!

/tmp/ipykernel_15844/3385605691.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


### range(len(test_word_split) 로 해보기 - X

In [95]:
def test_result4(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")

In [96]:
test_result4("돌체라떼 너무 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/2997558423.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [97]:
test_result4("돌체라떼 미친놈", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/2997558423.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### 값을 낮춰보기 - O

In [99]:
def test_result5(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [100]:
test_result5("돌체라떼 너무 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/4085268211.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [101]:
test_result5("돌체라떼 미친놈", embedded_model, lstm_model)

/tmp/ipykernel_15844/4085268211.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


### range(len(test_word_split))로 바꿔보기 - O

In [103]:
def test_result6(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [104]:
test_result6("돌체라떼 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [105]:
test_result6("돌체라떼 미친놈", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [106]:
test_result6("마라탕", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


#### O = 어느정도 구분은 하지만 비속어가 아닌데도 비속어라고 판별하는 경우가 있음
#### 예를 들어 마라탕을 왜 비속어라고 분류하는지? 알 수 있는 방법이 없다..
#### len() 에 어떤것이 들어가야 하는지는 알아봐야한다. 